<a href="https://colab.research.google.com/github/SidAS-ai/VERTEX-Assignment-/blob/main/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dotenv

In [13]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.5 MB/s eta 0:00:00


In [12]:
# @title updated  version
import json
import time
from typing import Dict, List, Any, Optional, Tuple
import google.generativeai as genai
from datetime import datetime
import pandas as pd
import numpy as np
from dataclasses import dataclass

@dataclass
class MatchMetrics:
    """Data class to store match metrics for easier analysis"""
    industry_alignment: float = 0.0
    stage_match: float = 0.0
    funding_fit: float = 0.0
    business_model_alignment: float = 0.0
    strategic_alignment: float = 0.0
    team_fit: float = 0.0
    market_alignment: float = 0.0
    vision_compatibility: float = 0.0
    risk_profile_match: float = 0.0
    growth_trajectory_fit: float = 0.0
    overall_score: float = 0.0

class EnhancedFounderInvestorMatcher:
    """
    A sophisticated system for matching founders with potential investors using
    generative AI to analyze compatibility across multiple dimensions.
    """

    def __init__(self, api_key: Optional[str] = None, model_name: str = 'gemini-1.5-pro'):
        """
        Initialize the matcher with API credentials and model configuration

        Args:
            api_key: Gemini API key (optional if already configured in environment)
            model_name: The specific Gemini model to use
        """
        # Configure the Gemini API
        if api_key:
            genai.configure(api_key=api_key)
        else:
            # This assumes API key is configured in environment variables
            try:
                genai.configure()
            except Exception as e:
                raise ValueError(f"No API key provided and no environment configuration found: {e}")

        # Initialize the Gemini model
        try:
            self.model = genai.GenerativeModel(model_name)
            self.model_name = model_name
        except Exception as e:
            raise ValueError(f"Failed to initialize Gemini model '{model_name}': {e}")

        # Track usage for rate limiting
        self.request_count = 0
        self.last_request_time = 0

        # Define dimensional weights for match scoring
        self.dimension_weights = {
            "industry_alignment": 0.15,
            "stage_match": 0.15,
            "funding_fit": 0.15,
            "business_model_alignment": 0.10,
            "strategic_alignment": 0.15,
            "team_fit": 0.10,
            "market_alignment": 0.05,
            "vision_compatibility": 0.05,
            "risk_profile_match": 0.05,
            "growth_trajectory_fit": 0.05
        }

        # Validate weights sum to 1.0
        total_weight = sum(self.dimension_weights.values())
        if not 0.99 <= total_weight <= 1.01:  # Allow for floating point imprecision
            raise ValueError(f"Dimension weights must sum to 1.0, got {total_weight}")

    def _rate_limit(self) -> None:
        """
        Implement rate limiting to prevent API throttling
        15 requests per minute max for Gemini API
        """
        self.request_count += 1

        # Reset counter after a minute
        current_time = time.time()
        if current_time - self.last_request_time > 60:
            self.request_count = 1
            self.last_request_time = current_time

        # If we're approaching the limit, pause
        if self.request_count >= 14:
            sleep_time = 60 - (current_time - self.last_request_time)
            if sleep_time > 0:
                print(f"Rate limit approaching. Pausing for {sleep_time:.2f} seconds...")
                time.sleep(sleep_time)
                self.request_count = 1
                self.last_request_time = time.time()

    def analyze_compatibility(self, founder_data: Dict[str, Any], investor_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Use Gemini API to analyze the compatibility between a founder and an investor
        with enhanced metrics and comprehensive analysis

        Args:
            founder_data: Dictionary containing founder information
            investor_data: Dictionary containing investor information

        Returns:
            Dictionary with detailed compatibility analysis
        """
        # Apply rate limiting
        self._rate_limit()

        # Extract market data for additional context if available
        market_trend = founder_data.get('market_trend', 'Unknown')
        market_size = founder_data.get('market_size', 'Unknown')
        competitive_landscape = founder_data.get('competitive_landscape', 'Unknown')

        # Extract team information
        team_background = founder_data.get('team_background', 'Unknown')
        team_experience = founder_data.get('team_experience', 'Unknown')

        # Construct the enhanced prompt for Gemini
        prompt = f"""
        Analyze the compatibility between the following founder and investor profiles.
        Provide a comprehensive and data-driven assessment using today's startup investment criteria.

        FOUNDER PROFILE:
        - Company Name: {founder_data.get('name', 'Unnamed')}
        - Industry/Sector: {founder_data.get('industry', 'Unknown')}
        - Startup Stage: {founder_data.get('stage', 'Unknown')}
        - Funding Required: ${founder_data.get('funding_required', 'Unknown')}
        - Traction: {founder_data.get('traction', 'Unknown')}
        - Business Model: {founder_data.get('business_model', 'Unknown')}
        - Target Market: {founder_data.get('target_market', 'Unknown')}
        - Product Status: {founder_data.get('product_status', 'Unknown')}
        - Team Experience: {team_experience}
        - Team Background: {team_background}
        - Vision & Mission: {founder_data.get('vision', 'Unknown')}
        - USP (Unique Selling Proposition): {founder_data.get('usp', 'Unknown')}
        - Growth Strategy: {founder_data.get('growth_strategy', 'Unknown')}
        - Market Size: {market_size}
        - Market Trend: {market_trend}
        - Competitive Landscape: {competitive_landscape}
        - Exit Strategy: {founder_data.get('exit_strategy', 'Unknown')}
        - Current Challenges: {founder_data.get('challenges', 'Unknown')}

        INVESTOR PROFILE:
        - Name: {investor_data.get('name', 'Unnamed')}
        - Preferred Industries: {', '.join(investor_data.get('preferred_industries', ['Unknown']))}
        - Investment Range: ${investor_data.get('investment_range', {}).get('min', 'Unknown')} - ${investor_data.get('investment_range', {}).get('max', 'Unknown')}
        - Preferred Startup Stages: {', '.join(investor_data.get('preferred_stages', ['Unknown']))}
        - Investment Thesis: {investor_data.get('investment_thesis', 'Unknown')}
        - Investment Style: {investor_data.get('investment_style', 'Unknown')}
        - Past Investments: {', '.join(investor_data.get('past_investments', ['Unknown']))}
        - Value-Add Offerings: {investor_data.get('value_add', 'Unknown')}
        - Geographic Focus: {investor_data.get('geographic_focus', 'Unknown')}
        - Expected ROI Timeline: {investor_data.get('roi_timeline', 'Unknown')}
        - Risk Tolerance: {investor_data.get('risk_tolerance', 'Unknown')}
        - Decision Factors: {investor_data.get('decision_factors', 'Unknown')}
        - Board Seat Requirements: {investor_data.get('board_requirements', 'Unknown')}
        - Follow-on Investment Potential: {investor_data.get('follow_on_potential', 'Unknown')}
        - Investment Constraints: {investor_data.get('constraints', 'Unknown')}

        Evaluate compatibility across these ten dimensions:
        1. Industry alignment (0-100): How well the startup's industry aligns with investor's preferred sectors
        2. Investment stage match (0-100): Compatibility between startup's current stage and investor's preferred stages
        3. Funding amount fit (0-100): How well the requested funding matches the investor's typical investment range
        4. Business model alignment (0-100): How well the startup's business model fits the investor's portfolio preferences
        5. Strategic alignment with investment thesis (0-100): How well the startup fits with the investor's overall strategy
        6. Team fit (0-100): How well the founding team's experience and background align with investor expectations
        7. Market alignment (0-100): How attractive the startup's target market is to the investor
        8. Vision compatibility (0-100): How well the startup's vision aligns with investor's long-term outlook
        9. Risk profile match (0-100): Compatibility between the startup's risk profile and investor's risk tolerance
        10. Growth trajectory fit (0-100): How well the startup's projected growth aligns with investor expectations

        For each dimension, provide a numerical score (0-100) and a detailed 2-3 sentence justification.
        Then calculate an overall match score (0-100) with this weighting:
        - Industry alignment: 15%
        - Investment stage match: 15%
        - Funding amount fit: 15%
        - Business model alignment: 10%
        - Strategic alignment: 15%
        - Team fit: 10%
        - Market alignment: 5%
        - Vision compatibility: 5%
        - Risk profile match: 5%
        - Growth trajectory fit: 5%

        After the dimensional analysis, provide a comprehensive 3-paragraph summary that includes:
        1. Key strengths of the match
        2. Potential concerns or misalignments
        3. Strategic recommendations for the founder to improve investor appeal

        Return your response in this JSON format:
        {{
            "industry_alignment": {{"score": 0, "justification": ""}},
            "stage_match": {{"score": 0, "justification": ""}},
            "funding_fit": {{"score": 0, "justification": ""}},
            "business_model_alignment": {{"score": 0, "justification": ""}},
            "strategic_alignment": {{"score": 0, "justification": ""}},
            "team_fit": {{"score": 0, "justification": ""}},
            "market_alignment": {{"score": 0, "justification": ""}},
            "vision_compatibility": {{"score": 0, "justification": ""}},
            "risk_profile_match": {{"score": 0, "justification": ""}},
            "growth_trajectory_fit": {{"score": 0, "justification": ""}},
            "overall_score": 0,
            "summary": "",
            "key_strengths": "",
            "potential_concerns": "",
            "strategic_recommendations": ""
        }}
        """

        try:
            # Call the Gemini API
            response = self.model.generate_content(prompt)

            # Extract the JSON from the response
            response_text = response.text

            # Find the JSON content
            if "```json" in response_text:
                json_content = response_text.split("```json")[1].split("```")[0].strip()
            elif "```" in response_text:
                json_content = response_text.split("```")[1].strip()
            else:
                json_content = response_text.strip()

            # Clean up any potential formatting issues
            json_content = json_content.replace('\n', ' ').replace('\r', '')

            try:
                result = json.loads(json_content)
                return result
            except json.JSONDecodeError as json_err:
                print(f"JSON parsing error: {json_err}")
                print(f"Attempted to parse: {json_content[:100]}...")
                # Return a simplified response when JSON parsing fails
                return self._generate_fallback_analysis(founder_data, investor_data)

        except Exception as e:
            print(f"Error with Gemini API: {e}")
            # Return a fallback analysis
            return self._generate_fallback_analysis(founder_data, investor_data)

    def _generate_fallback_analysis(self, founder_data: Dict[str, Any], investor_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Generate a fallback analysis when the AI-based analysis fails

        Args:
            founder_data: Dictionary containing founder information
            investor_data: Dictionary containing investor information

        Returns:
            Dictionary with basic compatibility analysis
        """
        # Basic scoring logic for industry match
        industry_score = 0
        if 'industry' in founder_data and 'preferred_industries' in investor_data:
            founder_industry = founder_data['industry'].lower()
            for ind in investor_data['preferred_industries']:
                if ind.lower() in founder_industry or founder_industry in ind.lower():
                    industry_score = 70  # Basic match
                    break

        # Basic scoring for stage match
        stage_score = 0
        if 'stage' in founder_data and 'preferred_stages' in investor_data:
            if founder_data['stage'] in investor_data['preferred_stages']:
                stage_score = 80

        # Basic scoring for funding fit
        funding_score = 0
        if 'funding_required' in founder_data and 'investment_range' in investor_data:
            funding = founder_data['funding_required']
            min_inv = investor_data['investment_range']['min']
            max_inv = investor_data['investment_range']['max']

            if min_inv <= funding <= max_inv:
                funding_score = 90
            elif funding < min_inv:
                # Too small
                funding_score = 40
            else:
                # Too large
                funding_score = 30

        # Calculate overall score using a subset of dimensions
        overall_score = (
            0.33 * industry_score +
            0.33 * stage_score +
            0.34 * funding_score
        )

        return {
            "industry_alignment": {"score": industry_score, "justification": "Basic industry keyword match analysis."},
            "stage_match": {"score": stage_score, "justification": "Direct comparison of stages."},
            "funding_fit": {"score": funding_score, "justification": "Simple range check for funding requirements."},
            "business_model_alignment": {"score": 50, "justification": "Default neutral score due to fallback mode."},
            "strategic_alignment": {"score": 50, "justification": "Default neutral score due to fallback mode."},
            "team_fit": {"score": 50, "justification": "Default neutral score due to fallback mode."},
            "market_alignment": {"score": 50, "justification": "Default neutral score due to fallback mode."},
            "vision_compatibility": {"score": 50, "justification": "Default neutral score due to fallback mode."},
            "risk_profile_match": {"score": 50, "justification": "Default neutral score due to fallback mode."},
            "growth_trajectory_fit": {"score": 50, "justification": "Default neutral score due to fallback mode."},
            "overall_score": overall_score,
            "summary": "This is a fallback analysis generated due to an error with the AI processing. It provides a basic assessment based on industry match, stage compatibility, and funding requirements.",
            "key_strengths": "Basic compatibility assessment only due to processing limitations.",
            "potential_concerns": "Unable to provide detailed concerns due to analysis limitations.",
            "strategic_recommendations": "Consider reviewing the match manually for better insights."
        }

    def match_founders_with_investors(
        self,
        founder_data: Dict[str, Any],
        investors_data: List[Dict[str, Any]],
        top_n: int = None
    ) -> List[Dict[str, Any]]:
        """
        Match a founder with multiple investors and rank the results

        Args:
            founder_data: Dictionary containing founder information
            investors_data: List of dictionaries containing investor information
            top_n: Optional limit to return only top N matches

        Returns:
            List of investors with match scores, sorted by score in descending order
        """
        results = []
        total_investors = len(investors_data)

        print(f"Starting compatibility analysis with {total_investors} potential investors...")

        for idx, investor in enumerate(investors_data, 1):
            print(f"Analyzing investor {idx}/{total_investors}: {investor['name']}...")

            # Get compatibility analysis for this founder-investor pair
            compatibility = self.analyze_compatibility(founder_data, investor)

            # Extract key insights
            key_strengths = compatibility.get("key_strengths", "No specific strengths identified")
            potential_concerns = compatibility.get("potential_concerns", "No specific concerns identified")
            strategic_recommendations = compatibility.get("strategic_recommendations", "No specific recommendations")

            # Add to results
            results.append({
                "investor_name": investor["name"],
                "match_score": compatibility.get("overall_score", 0),
                "match_summary": compatibility.get("summary", "Analysis failed"),
                "key_strengths": key_strengths,
                "potential_concerns": potential_concerns,
                "strategic_recommendations": strategic_recommendations,
                "detailed_analysis": {
                    "industry_alignment": compatibility.get("industry_alignment", {"score": 0, "justification": "Analysis failed"}),
                    "stage_match": compatibility.get("stage_match", {"score": 0, "justification": "Analysis failed"}),
                    "funding_fit": compatibility.get("funding_fit", {"score": 0, "justification": "Analysis failed"}),
                    "business_model_alignment": compatibility.get("business_model_alignment", {"score": 0, "justification": "Analysis failed"}),
                    "strategic_alignment": compatibility.get("strategic_alignment", {"score": 0, "justification": "Analysis failed"}),
                    "team_fit": compatibility.get("team_fit", {"score": 0, "justification": "Analysis failed"}),
                    "market_alignment": compatibility.get("market_alignment", {"score": 0, "justification": "Analysis failed"}),
                    "vision_compatibility": compatibility.get("vision_compatibility", {"score": 0, "justification": "Analysis failed"}),
                    "risk_profile_match": compatibility.get("risk_profile_match", {"score": 0, "justification": "Analysis failed"}),
                    "growth_trajectory_fit": compatibility.get("growth_trajectory_fit", {"score": 0, "justification": "Analysis failed"})
                },
                "investor_profile": investor
            })

        # Sort by match score descending
        results.sort(key=lambda x: x["match_score"], reverse=True)

        # Limit to top N if specified
        if top_n and isinstance(top_n, int) and top_n > 0:
            results = results[:top_n]

        return results

    def generate_match_report(
        self,
        founder_data: Dict[str, Any],
        match_results: List[Dict[str, Any]],
        output_format: str = 'markdown'
    ) -> str:
        """
        Generate a comprehensive report of the matching results

        Args:
            founder_data: Dictionary containing founder information
            match_results: The results from match_founders_with_investors
            output_format: 'markdown' or 'html'

        Returns:
            String containing the formatted report
        """
        timestamp = datetime.now().strftime("%B %d, %Y - %H:%M")

        if output_format == 'markdown':
            report = f"# Founder-Investor Match Report\n\n"
            report += f"**Generated:** {timestamp}\n\n"

            report += "## Startup Profile\n\n"
            report += f"**Company:** {founder_data.get('name', 'Unnamed Startup')}\n"
            report += f"**Industry:** {founder_data.get('industry', 'N/A')}\n"
            report += f"**Stage:** {founder_data.get('stage', 'N/A')}\n"
            report += f"**Funding Required:** \${founder_data.get('funding_required', 'N/A')}\n"
            report += f"**Business Model:** {founder_data.get('business_model', 'N/A')}\n\n"

            report += "## Top Investor Matches\n\n"

            for i, match in enumerate(match_results, 1):
                report += f"### {i}. {match['investor_name']} - Match Score: {match['match_score']:.1f}%\n\n"
                report += f"**Match Summary:**\n{match['match_summary']}\n\n"

                report += "**Key Strengths:**\n"
                report += f"{match['key_strengths']}\n\n"

                report += "**Potential Concerns:**\n"
                report += f"{match['potential_concerns']}\n\n"

                report += "**Strategic Recommendations:**\n"
                report += f"{match['strategic_recommendations']}\n\n"

                report += "**Dimensional Analysis:**\n\n"

                # Create a table for the dimensional analysis
                report += "| Dimension | Score | Justification |\n"
                report += "|-----------|-------|---------------|\n"

                for dim, details in match['detailed_analysis'].items():
                    dim_name = dim.replace('_', ' ').title()
                    score = details['score']
                    justification = details['justification']
                    report += f"| {dim_name} | {score:.1f} | {justification} |\n"

                report += "\n---\n\n"

        elif output_format == 'html':
            # HTML implementation would go here
            report = "<html><body><h1>HTML report generation not implemented yet</h1></body></html>"

        else:
            report = "Unsupported output format requested."

        return report

    def analyze_investor_compatibility_matrix(self, match_results: List[Dict[str, Any]]) -> pd.DataFrame:
        """
        Create a compatibility matrix to visualize strengths and weaknesses across investors

        Args:
            match_results: The results from match_founders_with_investors

        Returns:
            Pandas DataFrame with dimensional scores for each investor
        """
        # Extract the dimensions and scores
        data = []
        for match in match_results:
            row = {'investor': match['investor_name'], 'overall_score': match['match_score']}

            for dim, details in match['detailed_analysis'].items():
                row[dim] = details['score']

            data.append(row)

        # Create a DataFrame
        df = pd.DataFrame(data)

        # Set the investor name as the index
        if 'investor' in df.columns:
            df.set_index('investor', inplace=True)

        return df

    def recommend_investor_approach(self, investor_match: Dict[str, Any]) -> str:
        """
        Generate a tailored approach strategy for a specific investor based on the match analysis

        Args:
            investor_match: A single match result from the match_founders_with_investors output

        Returns:
            String containing approach recommendations
        """
        investor_name = investor_match['investor_name']
        profile = investor_match['investor_profile']
        analysis = investor_match['detailed_analysis']

        # Identify strengths (dimensions with scores > 75)
        strengths = []
        for dim, details in analysis.items():
            if details['score'] > 75:
                strengths.append(dim.replace('_', ' ').title())

        # Identify weaknesses (dimensions with scores < 50)
        weaknesses = []
        for dim, details in analysis.items():
            if details['score'] < 50:
                weaknesses.append((dim.replace('_', ' ').title(), details['justification']))

        # Generate the approach strategy
        strategy = f"## Approach Strategy for {investor_name}\n\n"

        # Lead with strengths
        strategy += "### Lead With These Strengths\n\n"
        if strengths:
            for strength in strengths:
                strategy += f"- **{strength}**: Emphasize this as a key selling point\n"
        else:
            strategy += "- No particularly strong match points identified. Consider a more general approach.\n"

        # Address potential concerns
        strategy += "\n### Prepare to Address These Concerns\n\n"
        if weaknesses:
            for weak, justification in weaknesses:
                strategy += f"- **{weak}**: {justification}\n"
        else:
            strategy += "- No significant concerns identified.\n"

        # Personalization tips
        strategy += "\n### Personalization Tips\n\n"

        # Reference past investments if available
        if 'past_investments' in profile and profile['past_investments']:
            strategy += "- Reference their investments in: " + ", ".join(profile['past_investments'][:3]) + "\n"

        # Reference their thesis
        if 'investment_thesis' in profile and profile['investment_thesis']:
            strategy += f"- Connect your vision to their thesis: \"{profile['investment_thesis'][:100]}...\"\n"

        # Value-add alignment
        if 'value_add' in profile and profile['value_add']:
            strategy += f"- Request specific help with: {profile['value_add']}\n"

        return strategy

class FounderInvestorMatchingPlatform:
    """
    Comprehensive platform for founder-investor matching with additional analysis tools,
    reporting capabilities, and integration features.
    """

    def __init__(self, api_key: Optional[str] = None, model_name: str = 'gemini-1.5-pro'):
        """
        Initialize the platform with the core matcher and additional features

        Args:
            api_key: Gemini API key
            model_name: The specific model to use
        """
        self.matcher = EnhancedFounderInvestorMatcher(api_key, model_name)
        self.match_history = []
        self.investor_database = {}
        self.founder_database = {}

    def add_investors_to_database(self, investors: List[Dict[str, Any]]) -> None:
        """
        Add investors to the platform's database for future matching

        Args:
            investors: List of investor profiles
        """
        for investor in investors:
            if 'name' in investor:
                self.investor_database[investor['name']] = investor

    def add_founders_to_database(self, founders: List[Dict[str, Any]]) -> None:
        """
        Add founders to the platform's database for future matching

        Args:
            founders: List of founder profiles
        """
        for founder in founders:
            if 'name' in founder:
                self.founder_database[founder['name']] = founder

    def find_match(
        self,
        founder_name: str,
        top_n: int = 3,
        custom_investors: List[Dict[str, Any]] = None
    ) -> Dict[str, Any]:
        """
        Find matches for a specific founder in the database

        Args:
            founder_name: Name of the founder to match
            top_n: Number of top matches to return
            custom_investors: Optional custom list of investors to match against

        Returns:
            Dictionary with match results and analysis
        """
        if founder_name not in self.founder_database:
            raise ValueError(f"Founder '{founder_name}' not found in database")

        founder_data = self.founder_database[founder_name]

        # Determine which investors to match against
        if custom_investors is not None:
            investors_to_match = custom_investors
        else:
            investors_to_match = list(self.investor_database.values())

        # Perform the matching
        match_results = self.matcher.match_founders_with_investors(
            founder_data,
            investors_to_match,
            top_n=top_n
        )

        # Generate additional analysis
        compatibility_matrix = self.matcher.analyze_investor_compatibility_matrix(match_results)

        # Generate the match report
        match_report = self.matcher.generate_match_report(founder_data, match_results)

        # Create approach strategies for the top matches
        approach_strategies = {}
        for match in match_results:
            investor_name = match['investor_name']
            approach_strategies[investor_name] = self.matcher.recommend_investor_approach(match)

        # Store the match results in history
        match_record = {
            'timestamp': datetime.now().isoformat(),
            'founder': founder_name,
            'investors_matched': len(investors_to_match),
            'top_matches': [m['investor_name'] for m in match_results],
            'match_results': match_results
        }
        self.match_history.append(match_record)

        # Return the comprehensive results
        return {
            'founder': founder_data,
            'match_results': match_results,
            'compatibility_matrix': compatibility_matrix,
            'match_report': match_report,
            'approach_strategies': approach_strategies
        }

    def batch_match_founders(
        self,
        founder_names: List[str],
        investors_per_founder: int = 3
    ) -> Dict[str, Dict[str, Any]]:
        """
        Perform batch matching for multiple founders

        Args:
            founder_names: List of founder names to match
            investors_per_founder: Number of investors to match per founder

        Returns:
            Dictionary mapping founder names to their match results
        """
        batch_results = {}

        for founder_name in founder_names:
            try:
                match_result = self.find_match(founder_name, top_n=investors_per_founder)
                batch_results[founder_name] = match_result
            except Exception as e:
                print(f"Error matching founder '{founder_name}': {e}")
                batch_results[founder_name] = {"error": str(e)}

        return batch_results

    def generate_investor_insights(self, investor_name: str) -> Dict[str, Any]:
        """
        Generate insights about a specific investor based on past matches

        Args:
            investor_name: Name of the investor to analyze

        Returns:
            Dictionary with insights about the investor
        """
        if investor_name not in self.investor_database:
            raise ValueError(f"Investor '{investor_name}' not found in database")

        investor_profile = self.investor_database[investor_name]

        # Find all matches involving this investor
        relevant_matches = []
        for match_record in self.match_history:
            for match in match_record['match_results']:
                if match['investor_name'] == investor_name:
                    match_with_founder = {
                        'founder': match_record['founder'],
                        'score': match['match_score'],
                        'timestamp': match_record['timestamp'],
                        'detailed_analysis': match['detailed_analysis']
                    }
                    relevant_matches.append(match_with_founder)

        # Analyze match patterns
        if relevant_matches:
            # Calculate average scores across dimensions
            dimension_scores = {
                'industry_alignment': [],
                'stage_match': [],
                'funding_fit': [],
                'business_model_alignment': [],
                'strategic_alignment': [],
                'team_fit': [],
                'market_alignment': [],
                'vision_compatibility': [],
                'risk_profile_match': [],
                'growth_trajectory_fit': []
            }

            for match in relevant_matches:
                for dim, values in dimension_scores.items():
                    if dim in match['detailed_analysis']:
                        values.append(match['detailed_analysis'][dim]['score'])

            # Calculate averages for each dimension
            dimension_averages = {}
            for dim, scores in dimension_scores.items():
                if scores:
                    dimension_averages[dim] = sum(scores) / len(scores)
                else:
                    dimension_averages[dim] = 0

            # Find strongest and weakest dimensions
            sorted_dimensions = sorted(dimension_averages.items(), key=lambda x: x[1], reverse=True)
            strongest_dimensions = sorted_dimensions[:3]
            weakest_dimensions = sorted_dimensions[-3:]

            # Calculate match trend over time
            matches_by_time = sorted(relevant_matches, key=lambda x: x['timestamp'])
            match_trend = [
                {'timestamp': m['timestamp'], 'score': m['score']}
                for m in matches_by_time
            ]

            insights = {
                'investor_profile': investor_profile,
                'match_count': len(relevant_matches),
                'average_match_score': sum(m['score'] for m in relevant_matches) / len(relevant_matches),
                'dimension_averages': dimension_averages,
                'strongest_dimensions': strongest_dimensions,
                'weakest_dimensions': weakest_dimensions,
                'match_trend': match_trend,
                'founder_matches': [m['founder'] for m in relevant_matches]
            }
        else:
            insights = {
                'investor_profile': investor_profile,
                'match_count': 0,
                'message': "No match history found for this investor"
            }

        return insights

    def export_match_data(self, format: str = 'json') -> Any:
        """
        Export all match data in the specified format

        Args:
            format: 'json', 'csv', or 'pandas'

        Returns:
            Data in the specified format
        """
        if format == 'json':
            return json.dumps(self.match_history, indent=2)

        elif format == 'pandas':
            # Create a flattened dataframe
            records = []
            for match_record in self.match_history:
                founder = match_record['founder']
                timestamp = match_record['timestamp']

                for match in match_record['match_results']:
                    record = {
                        'timestamp': timestamp,
                        'founder': founder,
                        'investor': match['investor_name'],
                        'match_score': match['match_score']
                    }

                    # Add dimension scores
                    for dim, details in match['detailed_analysis'].items():
                        record[f'{dim}_score'] = details['score']

                    records.append(record)

            return pd.DataFrame(records)

        elif format == 'csv':
            df = self.export_match_data(format='pandas')
            return df.to_csv(index=False)

        else:
            raise ValueError(f"Unsupported export format: {format}")

    def get_market_insights(self) -> Dict[str, Any]:
        """
        Analyze patterns across all matches to generate market insights

        Returns:
            Dictionary with market insights
        """
        if not self.match_history:
            return {"message": "No match data available for analysis"}

        # Prepare data for analysis
        all_matches = []
        for record in self.match_history:
            founder = record['founder']
            founder_industry = self.founder_database.get(founder, {}).get('industry', 'Unknown')
            founder_stage = self.founder_database.get(founder, {}).get('stage', 'Unknown')

            for match in record['match_results']:
                all_matches.append({
                    'founder': founder,
                    'industry': founder_industry,
                    'stage': founder_stage,
                    'investor': match['investor_name'],
                    'score': match['match_score']
                })

        matches_df = pd.DataFrame(all_matches)

        # Industry analysis
        industry_analysis = {}
        if 'industry' in matches_df.columns:
            avg_scores_by_industry = matches_df.groupby('industry')['score'].mean().sort_values(ascending=False)
            match_counts_by_industry = matches_df.groupby('industry').size().sort_values(ascending=False)

            industry_analysis = {
                'average_scores': avg_scores_by_industry.to_dict(),
                'match_counts': match_counts_by_industry.to_dict(),
                'top_industries': avg_scores_by_industry.head(3).index.tolist(),
                'bottom_industries': avg_scores_by_industry.tail(3).index.tolist()
            }

        # Stage analysis
        stage_analysis = {}
        if 'stage' in matches_df.columns:
            avg_scores_by_stage = matches_df.groupby('stage')['score'].mean().sort_values(ascending=False)
            match_counts_by_stage = matches_df.groupby('stage').size().sort_values(ascending=False)

            stage_analysis = {
                'average_scores': avg_scores_by_stage.to_dict(),
                'match_counts': match_counts_by_stage.to_dict(),
                'top_stages': avg_scores_by_stage.head(3).index.tolist(),
                'bottom_stages': avg_scores_by_stage.tail(3).index.tolist()
            }

        # Investor analysis
        investor_analysis = {}
        if 'investor' in matches_df.columns:
            avg_scores_by_investor = matches_df.groupby('investor')['score'].mean().sort_values(ascending=False)
            match_counts_by_investor = matches_df.groupby('investor').size().sort_values(ascending=False)

            investor_analysis = {
                'average_scores': avg_scores_by_investor.to_dict(),
                'match_counts': match_counts_by_investor.to_dict(),
                'most_compatible_investors': avg_scores_by_investor.head(3).index.tolist(),
                'least_compatible_investors': avg_scores_by_investor.tail(3).index.tolist(),
                'most_active_investors': match_counts_by_investor.head(3).index.tolist()
            }

        # Return comprehensive insights
        return {
            'industry_analysis': industry_analysis,
            'stage_analysis': stage_analysis,
            'investor_analysis': investor_analysis,
            'total_matches_analyzed': len(all_matches),
            'average_match_score': matches_df['score'].mean() if 'score' in matches_df.columns else 0,
            'generated_at': datetime.now().isoformat()
        }

# Example usage with enhanced functionality
def main():
    """Run a comprehensive demo of the founder-investor matching platform"""

    # Get API key from environment or user input
    # import os
    # gemini_api_key = os.getenv("GEMINI_API_KEY")
    gemini_api_key = ""  # Example only

    try:
        # Initialize the platform
        print("Initializing the FounderInvestorMatchingPlatform...")
        platform = FounderInvestorMatchingPlatform(api_key=gemini_api_key)

        # Load sample data
        founder_data = get_sample_founder_data()
        investors_data = get_sample_investors_data()

        # Add to platform database
        platform.add_founders_to_database([founder_data])
        platform.add_investors_to_database(investors_data)

        print(f"\nPlatform initialized with {len(platform.founder_database)} founders and {len(platform.investor_database)} investors")

        # Print available models to help with troubleshooting
        try:
            print("\nAvailable Gemini models:")
            for model in genai.list_models():
                print(f" - {model.name}")
        except Exception as e:
            print(f"Could not list models: {e}")

        # Run a match
        print("\nRunning intelligent founder-investor matching...")
        company_name = founder_data['name']
        match_results = platform.find_match(company_name, top_n=3)

        # Display results
        print("\n=== FOUNDER-INVESTOR MATCHING RESULTS ===\n")
        print(f"Founder: {company_name} ({founder_data['industry']}, {founder_data['stage']} stage)")
        print("\nMatched Investors (ranked by compatibility):\n")

        for i, match in enumerate(match_results['match_results'], 1):
            print(f"{i}. {match['investor_name']}")
            print(f"   Match Score: {match['match_score']:.1f}/100")
            print(f"   Summary: {match['match_summary'][:200]}...")
            print()

        # Show the match report for the top investor
        print("\n=== DETAILED MATCH REPORT ===\n")
        print(match_results['match_report'][:500] + "...\n")

        # Show approach strategy
        top_investor = match_results['match_results'][0]['investor_name']
        print(f"\n=== APPROACH STRATEGY FOR {top_investor} ===\n")
        print(match_results['approach_strategies'][top_investor][:500] + "...\n")

        # Interactive mode
        while True:
            print("\nOptions:")
            print("1. View detailed analysis for a specific investor")
            print("2. Export match data")
            print("3. View market insights")
            print("4. Exit")

            choice = input("\nEnter your choice (1-4): ")

            if choice == '1':
                investor_idx = int(input(f"Choose investor (1-{len(match_results['match_results'])}): ")) - 1
                if 0 <= investor_idx < len(match_results['match_results']):
                    investor = match_results['match_results'][investor_idx]['investor_name']
                    print(f"\n=== DETAILED ANALYSIS: {company_name} + {investor} ===\n")

                    analysis = match_results['match_results'][investor_idx]['detailed_analysis']
                    for category, details in analysis.items():
                        print(f"{category.replace('_', ' ').title()}: {details['score']:.1f}/100")
                        print(f"  {details['justification']}")
                        print()
                else:
                    print("Invalid selection.")

            elif choice == '2':
                format_choice = input("Export format (json, csv, pandas): ").lower()
                if format_choice in ['json', 'csv', 'pandas']:
                    data = platform.export_match_data(format=format_choice)
                    if format_choice == 'pandas':
                        print(data.head())
                    else:
                        print(f"Data exported as {format_choice} ({len(data)} bytes)")
                else:
                    print("Invalid format.")

            elif choice == '3':
                insights = platform.get_market_insights()
                print("\n=== MARKET INSIGHTS ===\n")

                if 'industry_analysis' in insights and insights['industry_analysis']:
                    print("Top Industries by Match Score:")
                    for industry in insights['industry_analysis'].get('top_industries', []):
                        score = insights['industry_analysis']['average_scores'].get(industry, 0)
                        print(f"  - {industry}: {score:.1f}/100")

                if 'investor_analysis' in insights and insights['investor_analysis']:
                    print("\nMost Compatible Investors:")
                    for investor in insights['investor_analysis'].get('most_compatible_investors', []):
                        score = insights['investor_analysis']['average_scores'].get(investor, 0)
                        print(f"  - {investor}: {score:.1f}/100")

                print(f"\nAverage Match Score Across All Matches: {insights.get('average_match_score', 0):.1f}/100")

            elif choice == '4':
                print("Exiting the platform. Thank you!")
                break

            else:
                print("Invalid choice. Please select 1-4.")

    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

# Example of enhanced founder data structure
def get_sample_founder_data() -> Dict[str, Any]:
    return {
        "name": "TechInnovate AI",
        "industry": "Artificial Intelligence / Machine Learning",
        "stage": "Seed",
        "funding_required": 750000,
        "traction": "5,000 monthly active users, \$15,000 MRR, 12% MoM growth",
        "business_model": "B2B SaaS - Subscription",
        "target_market": "Mid-sized enterprises in healthcare, finance, and retail",
        "product_status": "MVP with 15 paying customers",
        "team_background": "Founded by ex-Google AI researcher and Stanford CS PhD",
        "team_experience": "Combined 12 years in AI/ML, previous startup exit of \$8M",
        "vision": "Democratize access to AI tools for businesses of all sizes",
        "usp": "Custom AI models with 60% less training data than competitors",
        "growth_strategy": "Land and expand within enterprise accounts, strategic partnerships",
        "market_size": "\$15B TAM, growing at 35% CAGR",
        "market_trend": "Increasing demand for specialized AI solutions in regulated industries",
        "competitive_landscape": "Fragmented market with no dominant player in our niche",
        "exit_strategy": "Acquisition by major cloud provider or enterprise software company within 5-7 years",
        "challenges": "Scaling sales team, managing increasing cloud compute costs"
    }

# Enhanced investor data structure
def get_sample_investors_data() -> List[Dict[str, Any]]:
    return [
        {
            "name": "Alpha Ventures",
            "preferred_industries": ["AI", "Machine Learning", "SaaS", "Enterprise Software"],
            "investment_range": {"min": 500000, "max": 2000000},
            "preferred_stages": ["Seed", "Pre-seed"],
            "investment_thesis": "Investing in AI-driven solutions that transform traditional industries with a focus on enterprise applications",
            "investment_style": "Hands-on, weekly check-ins, active board participation",
            "past_investments": ["DataRobot", "ScaleAI", "Hugging Face"],
            "value_add": "AI talent recruitment, enterprise sales connections, cloud credits",
            "geographic_focus": "North America, Europe",
            "roi_timeline": "5-7 years",
            "risk_tolerance": "Moderate - expects 1 in 5 investments to drive returns",
            "decision_factors": "Team expertise, technology differentiation, market opportunity",
            "board_requirements": "Observer seat at minimum",
            "follow_on_potential": "Up to \$5M in follow-on funding for top performers",
            "constraints": "No hardware companies, no pure consumer plays"
        },
        {
            "name": "Beta Capital",
            "preferred_industries": ["Fintech", "Blockchain", "Crypto", "AI in Finance"],
            "investment_range": {"min": 1000000, "max": 5000000},
            "preferred_stages": ["Series A"],
            "investment_thesis": "Supporting financial technology innovation with a focus on infrastructure and B2B solutions",
            "investment_style": "Capital-focused, quarterly reviews, limited operational involvement",
            "past_investments": ["Stripe", "Plaid", "Coinbase"],
            "value_add": "Financial services partnerships, regulatory guidance, M&A support",
            "geographic_focus": "Global",
            "roi_timeline": "4-6 years",
            "risk_tolerance": "Conservative - focuses on proven business models",
            "decision_factors": "Revenue growth, unit economics, regulatory compliance",
            "board_requirements": "Board seat for investments over \$2M",
            "follow_on_potential": "Participates pro-rata in future rounds",
            "constraints": "Must have \$500K+ ARR for Series A"
        },
        {
            "name": "Gamma Partners",
            "preferred_industries": ["Healthcare", "Biotech", "AI in Healthcare"],
            "investment_range": {"min": 250000, "max": 1000000},
            "preferred_stages": ["Pre-seed", "Seed"],
            "investment_thesis": "Backing early-stage startups using technology to improve healthcare outcomes",
            "investment_style": "Founder-friendly, supportive, monthly advisory calls",
            "past_investments": ["Tempus", "Recursion", "PathAI"],
            "value_add": "Clinical trial design support, FDA approval guidance, hospital network access",
            "geographic_focus": "North America",
            "roi_timeline": "7-10 years",
            "risk_tolerance": "High - willing to back moonshot ideas",
            "decision_factors": "Novel IP, clinical value proposition, team scientific credibility",
            "board_requirements": "No formal requirements at seed stage",
            "follow_on_potential": "Selective follow-on strategy",
            "constraints": "Requires at least one healthcare domain expert on founding team"
        },
        {
            "name": "Delta Seed Fund",
            "preferred_industries": ["AI", "Data Infrastructure", "Developer Tools", "Enterprise SaaS"],
            "investment_range": {"min": 400000, "max": 1500000},
            "preferred_stages": ["Pre-seed", "Seed"],
            "investment_thesis": "Supporting technical founders building foundational technology and tools for the AI-driven economy",
            "investment_style": "Technical support, founder-centric, biweekly office hours",
            "past_investments": ["Databricks", "Snorkel AI", "Weights & Biases"],
            "value_add": "Technical architecture review, open source strategy, dev community building",
            "geographic_focus": "North America, Europe, Israel",
            "roi_timeline": "5-8 years",
            "risk_tolerance": "Moderate-High - believes in technical differentiation",
            "decision_factors": "Technical innovation, IP potential, founder technical credibility",
            "board_requirements": "Advisory role only at seed stage",
            "follow_on_potential": "Reserves 2x initial investment for follow-on",
            "constraints": "Founders must have technical background"
        },
        {
            "name": "Epsilon Growth Capital",
            "preferred_industries": ["AI Applications", "Vertical SaaS", "Industry-Specific Software"],
            "investment_range": {"min": 1000000, "max": 3000000},
            "preferred_stages": ["Seed", "Series A"],
            "investment_thesis": "Scaling proven AI applications with industry-specific focus to achieve rapid market penetration",
            "investment_style": "Growth-focused, operational support, sales acceleration",
            "past_investments": ["Gong.io", "UiPath", "Canva"],
            "value_add": "Enterprise sales strategy, pricing optimization, go-to-market support",
            "geographic_focus": "North America, Europe, Australia",
            "roi_timeline": "4-6 years",
            "risk_tolerance": "Moderate - prefers proven product-market fit",
            "decision_factors": "Revenue growth rate, CAC/LTV, market expansion potential",
            "board_requirements": "Board seat for investments over \$1.5M",
            "follow_on_potential": "Strong follow-on capabilities up to \$10M total per company",
            "constraints": "Must demonstrate clear product-market fit with at least 10 paying customers"
        }
    ]

if __name__ == "__main__":
    main()


Initializing the FounderInvestorMatchingPlatform...

Platform initialized with 1 founders and 5 investors

Available Gemini models:
 - models/chat-bison-001
 - models/text-bison-001
 - models/embedding-gecko-001
 - models/gemini-1.0-pro-vision-latest
 - models/gemini-pro-vision
 - models/gemini-1.5-pro-latest
 - models/gemini-1.5-pro-001
 - models/gemini-1.5-pro-002
 - models/gemini-1.5-pro
 - models/gemini-1.5-flash-latest
 - models/gemini-1.5-flash-001
 - models/gemini-1.5-flash-001-tuning
 - models/gemini-1.5-flash
 - models/gemini-1.5-flash-002
 - models/gemini-1.5-flash-8b
 - models/gemini-1.5-flash-8b-001
 - models/gemini-1.5-flash-8b-latest
 - models/gemini-1.5-flash-8b-exp-0827
 - models/gemini-1.5-flash-8b-exp-0924
 - models/gemini-2.0-flash-exp
 - models/gemini-2.0-flash
 - models/gemini-2.0-flash-001
 - models/gemini-2.0-flash-exp-image-generation
 - models/gemini-2.0-flash-lite-001
 - models/gemini-2.0-flash-lite
 - models/gemini-2.0-flash-lite-preview-02-05
 - models/gemin

Error with Gemini API: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Analyzing investor 5/5: Epsilon Growth Capital...


Error with Gemini API: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).

=== FOUNDER-INVESTOR MATCHING RESULTS ===

Founder: TechInnovate AI (Artificial Intelligence / Machine Learning, Seed stage)

Matched Investors (ranked by compatibility):

1. Gamma Partners
   Match Score: 84.0/100
   Summary: TechInnovate AI and Gamma Partners demonstrate a strong potential match.  The startup's stage, funding requirements, and strong technical team align well with Gamma Partners' investment criteria.  Add...

2. Alpha Ventures
   Match Score: 80.1/100
   Summary: This is a fallback analysis generated due to an error with the AI processing. It provides a basic assessment based on industry match, stage compatibility, and funding requirements....

3. Delta Seed Fund
   Match Score: 57.0/100
   Summary: This is a fallback analysis generated due to an error with the AI pro

KeyboardInterrupt: Interrupted by user

In [15]:
# @title  code with the UI gradio
import gradio as gr
from typing import Dict, List, Any, Optional, Tuple
import json
import time
from datetime import datetime
import pandas as pd
import numpy as np
from dataclasses import dataclass
import google.generativeai as genai

@dataclass
class MatchMetrics:
    """Data class to store match metrics for easier analysis"""
    industry_alignment: float = 0.0
    stage_match: float = 0.0
    funding_fit: float = 0.0
    business_model_alignment: float = 0.0
    strategic_alignment: float = 0.0
    team_fit: float = 0.0
    market_alignment: float = 0.0
    vision_compatibility: float = 0.0
    risk_profile_match: float = 0.0
    growth_trajectory_fit: float = 0.0
    overall_score: float = 0.0

class EnhancedFounderInvestorMatcher:
    """
    A sophisticated system for matching founders with potential investors using
    generative AI to analyze compatibility across multiple dimensions.
    """

    def __init__(self, api_key: Optional[str] = None, model_name: str = 'gemini-1.5-pro'):
        """
        Initialize the matcher with API credentials and model configuration

        Args:
            api_key: Gemini API key (optional if already configured in environment)
            model_name: The specific Gemini model to use
        """
        # Configure the Gemini API
        if api_key:
            genai.configure(api_key=api_key)
        else:
            # This assumes API key is configured in environment variables
            try:
                genai.configure()
            except Exception as e:
                raise ValueError(f"No API key provided and no environment configuration found: {e}")

        # Initialize the Gemini model
        try:
            self.model = genai.GenerativeModel(model_name)
            self.model_name = model_name
        except Exception as e:
            raise ValueError(f"Failed to initialize Gemini model '{model_name}': {e}")

        # Track usage for rate limiting
        self.request_count = 0
        self.last_request_time = 0

        # Define dimensional weights for match scoring
        self.dimension_weights = {
            "industry_alignment": 0.15,
            "stage_match": 0.15,
            "funding_fit": 0.15,
            "business_model_alignment": 0.10,
            "strategic_alignment": 0.15,
            "team_fit": 0.10,
            "market_alignment": 0.05,
            "vision_compatibility": 0.05,
            "risk_profile_match": 0.05,
            "growth_trajectory_fit": 0.05
        }

        # Validate weights sum to 1.0
        total_weight = sum(self.dimension_weights.values())
        if not 0.99 <= total_weight <= 1.01:  # Allow for floating point imprecision
            raise ValueError(f"Dimension weights must sum to 1.0, got {total_weight}")

    def _rate_limit(self) -> None:
        """
        Implement rate limiting to prevent API throttling
        15 requests per minute max for Gemini API
        """
        self.request_count += 1

        # Reset counter after a minute
        current_time = time.time()
        if current_time - self.last_request_time > 60:
            self.request_count = 1
            self.last_request_time = current_time

        # If we're approaching the limit, pause
        if self.request_count >= 14:
            sleep_time = 60 - (current_time - self.last_request_time)
            if sleep_time > 0:
                print(f"Rate limit approaching. Pausing for {sleep_time:.2f} seconds...")
                time.sleep(sleep_time)
                self.request_count = 1
                self.last_request_time = time.time()

    def analyze_compatibility(self, founder_data: Dict[str, Any], investor_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Use Gemini API to analyze the compatibility between a founder and an investor
        with enhanced metrics and comprehensive analysis

        Args:
            founder_data: Dictionary containing founder information
            investor_data: Dictionary containing investor information

        Returns:
            Dictionary with detailed compatibility analysis
        """
        # Apply rate limiting
        self._rate_limit()

        # Extract market data for additional context if available
        market_trend = founder_data.get('market_trend', 'Unknown')
        market_size = founder_data.get('market_size', 'Unknown')
        competitive_landscape = founder_data.get('competitive_landscape', 'Unknown')

        # Extract team information
        team_background = founder_data.get('team_background', 'Unknown')
        team_experience = founder_data.get('team_experience', 'Unknown')

        # Construct the enhanced prompt for Gemini
        prompt = f"""
        Analyze the compatibility between the following founder and investor profiles.
        Provide a comprehensive and data-driven assessment using today's startup investment criteria.

        FOUNDER PROFILE:
        - Company Name: {founder_data.get('name', 'Unnamed')}
        - Industry/Sector: {founder_data.get('industry', 'Unknown')}
        - Startup Stage: {founder_data.get('stage', 'Unknown')}
        - Funding Required: ${founder_data.get('funding_required', 'Unknown')}
        - Traction: {founder_data.get('traction', 'Unknown')}
        - Business Model: {founder_data.get('business_model', 'Unknown')}
        - Target Market: {founder_data.get('target_market', 'Unknown')}
        - Product Status: {founder_data.get('product_status', 'Unknown')}
        - Team Experience: {team_experience}
        - Team Background: {team_background}
        - Vision & Mission: {founder_data.get('vision', 'Unknown')}
        - USP (Unique Selling Proposition): {founder_data.get('usp', 'Unknown')}
        - Growth Strategy: {founder_data.get('growth_strategy', 'Unknown')}
        - Market Size: {market_size}
        - Market Trend: {market_trend}
        - Competitive Landscape: {competitive_landscape}
        - Exit Strategy: {founder_data.get('exit_strategy', 'Unknown')}
        - Current Challenges: {founder_data.get('challenges', 'Unknown')}

        INVESTOR PROFILE:
        - Name: {investor_data.get('name', 'Unnamed')}
        - Preferred Industries: {', '.join(investor_data.get('preferred_industries', ['Unknown']))}
        - Investment Range: ${investor_data.get('investment_range', {}).get('min', 'Unknown')} - ${investor_data.get('investment_range', {}).get('max', 'Unknown')}
        - Preferred Startup Stages: {', '.join(investor_data.get('preferred_stages', ['Unknown']))}
        - Investment Thesis: {investor_data.get('investment_thesis', 'Unknown')}
        - Investment Style: {investor_data.get('investment_style', 'Unknown')}
        - Past Investments: {', '.join(investor_data.get('past_investments', ['Unknown']))}
        - Value-Add Offerings: {investor_data.get('value_add', 'Unknown')}
        - Geographic Focus: {investor_data.get('geographic_focus', 'Unknown')}
        - Expected ROI Timeline: {investor_data.get('roi_timeline', 'Unknown')}
        - Risk Tolerance: {investor_data.get('risk_tolerance', 'Unknown')}
        - Decision Factors: {investor_data.get('decision_factors', 'Unknown')}
        - Board Seat Requirements: {investor_data.get('board_requirements', 'Unknown')}
        - Follow-on Investment Potential: {investor_data.get('follow_on_potential', 'Unknown')}
        - Investment Constraints: {investor_data.get('constraints', 'Unknown')}

        Evaluate compatibility across these ten dimensions:
        1. Industry alignment (0-100): How well the startup's industry aligns with investor's preferred sectors
        2. Investment stage match (0-100): Compatibility between startup's current stage and investor's preferred stages
        3. Funding amount fit (0-100): How well the requested funding matches the investor's typical investment range
        4. Business model alignment (0-100): How well the startup's business model fits the investor's portfolio preferences
        5. Strategic alignment with investment thesis (0-100): How well the startup fits with the investor's overall strategy
        6. Team fit (0-100): How well the founding team's experience and background align with investor expectations
        7. Market alignment (0-100): How attractive the startup's target market is to the investor
        8. Vision compatibility (0-100): How well the startup's vision aligns with investor's long-term outlook
        9. Risk profile match (0-100): Compatibility between the startup's risk profile and investor's risk tolerance
        10. Growth trajectory fit (0-100): How well the startup's projected growth aligns with investor expectations

        For each dimension, provide a numerical score (0-100) and a detailed 2-3 sentence justification.
        Then calculate an overall match score (0-100) with this weighting:
        - Industry alignment: 15%
        - Investment stage match: 15%
        - Funding amount fit: 15%
        - Business model alignment: 10%
        - Strategic alignment: 15%
        - Team fit: 10%
        - Market alignment: 5%
        - Vision compatibility: 5%
        - Risk profile match: 5%
        - Growth trajectory fit: 5%

        After the dimensional analysis, provide a comprehensive 3-paragraph summary that includes:
        1. Key strengths of the match
        2. Potential concerns or misalignments
        3. Strategic recommendations for the founder to improve investor appeal

        Return your response in this JSON format:
        {{
            "industry_alignment": {{"score": 0, "justification": ""}},
            "stage_match": {{"score": 0, "justification": ""}},
            "funding_fit": {{"score": 0, "justification": ""}},
            "business_model_alignment": {{"score": 0, "justification": ""}},
            "strategic_alignment": {{"score": 0, "justification": ""}},
            "team_fit": {{"score": 0, "justification": ""}},
            "market_alignment": {{"score": 0, "justification": ""}},
            "vision_compatibility": {{"score": 0, "justification": ""}},
            "risk_profile_match": {{"score": 0, "justification": ""}},
            "growth_trajectory_fit": {{"score": 0, "justification": ""}},
            "overall_score": 0,
            "summary": "",
            "key_strengths": "",
            "potential_concerns": "",
            "strategic_recommendations": ""
        }}
        """

        try:
            # Call the Gemini API
            response = self.model.generate_content(prompt)

            # Extract the JSON from the response
            response_text = response.text

            # Find the JSON content
            if "```json" in response_text:
                json_content = response_text.split("```json")[1].split("```")[0].strip()
            elif "```" in response_text:
                json_content = response_text.split("```")[1].strip()
            else:
                json_content = response_text.strip()

            # Clean up any potential formatting issues
            json_content = json_content.replace('\n', ' ').replace('\r', '')

            try:
                result = json.loads(json_content)
                return result
            except json.JSONDecodeError as json_err:
                print(f"JSON parsing error: {json_err}")
                print(f"Attempted to parse: {json_content[:100]}...")
                # Return a simplified response when JSON parsing fails
                return self._generate_fallback_analysis(founder_data, investor_data)

        except Exception as e:
            print(f"Error with Gemini API: {e}")
            # Return a fallback analysis
            return self._generate_fallback_analysis(founder_data, investor_data)

    def _generate_fallback_analysis(self, founder_data: Dict[str, Any], investor_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Generate a fallback analysis when the AI-based analysis fails

        Args:
            founder_data: Dictionary containing founder information
            investor_data: Dictionary containing investor information

        Returns:
            Dictionary with basic compatibility analysis
        """
        # Basic scoring logic for industry match
        industry_score = 0
        if 'industry' in founder_data and 'preferred_industries' in investor_data:
            founder_industry = founder_data['industry'].lower()
            for ind in investor_data['preferred_industries']:
                if ind.lower() in founder_industry or founder_industry in ind.lower():
                    industry_score = 70  # Basic match
                    break

        # Basic scoring for stage match
        stage_score = 0
        if 'stage' in founder_data and 'preferred_stages' in investor_data:
            if founder_data['stage'] in investor_data['preferred_stages']:
                stage_score = 80

        # Basic scoring for funding fit
        funding_score = 0
        if 'funding_required' in founder_data and 'investment_range' in investor_data:
            funding = founder_data['funding_required']
            min_inv = investor_data['investment_range']['min']
            max_inv = investor_data['investment_range']['max']

            if min_inv <= funding <= max_inv:
                funding_score = 90
            elif funding < min_inv:
                # Too small
                funding_score = 40
            else:
                # Too large
                funding_score = 30

        # Calculate overall score using a subset of dimensions
        overall_score = (
            0.33 * industry_score +
            0.33 * stage_score +
            0.34 * funding_score
        )

        return {
            "industry_alignment": {"score": industry_score, "justification": "Basic industry keyword match analysis."},
            "stage_match": {"score": stage_score, "justification": "Direct comparison of stages."},
            "funding_fit": {"score": funding_score, "justification": "Simple range check for funding requirements."},
            "business_model_alignment": {"score": 50, "justification": "Default neutral score due to fallback mode."},
            "strategic_alignment": {"score": 50, "justification": "Default neutral score due to fallback mode."},
            "team_fit": {"score": 50, "justification": "Default neutral score due to fallback mode."},
            "market_alignment": {"score": 50, "justification": "Default neutral score due to fallback mode."},
            "vision_compatibility": {"score": 50, "justification": "Default neutral score due to fallback mode."},
            "risk_profile_match": {"score": 50, "justification": "Default neutral score due to fallback mode."},
            "growth_trajectory_fit": {"score": 50, "justification": "Default neutral score due to fallback mode."},
            "overall_score": overall_score,
            "summary": "This is a fallback analysis generated due to an error with the AI processing. It provides a basic assessment based on industry match, stage compatibility, and funding requirements.",
            "key_strengths": "Basic compatibility assessment only due to processing limitations.",
            "potential_concerns": "Unable to provide detailed concerns due to analysis limitations.",
            "strategic_recommendations": "Consider reviewing the match manually for better insights."
        }

    def match_founders_with_investors(
        self,
        founder_data: Dict[str, Any],
        investors_data: List[Dict[str, Any]],
        top_n: int = None
    ) -> List[Dict[str, Any]]:
        """
        Match a founder with multiple investors and rank the results

        Args:
            founder_data: Dictionary containing founder information
            investors_data: List of dictionaries containing investor information
            top_n: Optional limit to return only top N matches

        Returns:
            List of investors with match scores, sorted by score in descending order
        """
        results = []
        total_investors = len(investors_data)

        print(f"Starting compatibility analysis with {total_investors} potential investors...")

        for idx, investor in enumerate(investors_data, 1):
            print(f"Analyzing investor {idx}/{total_investors}: {investor['name']}...")

            # Get compatibility analysis for this founder-investor pair
            compatibility = self.analyze_compatibility(founder_data, investor)

            # Extract key insights
            key_strengths = compatibility.get("key_strengths", "No specific strengths identified")
            potential_concerns = compatibility.get("potential_concerns", "No specific concerns identified")
            strategic_recommendations = compatibility.get("strategic_recommendations", "No specific recommendations")

            # Add to results
            results.append({
                "investor_name": investor["name"],
                "match_score": compatibility.get("overall_score", 0),
                "match_summary": compatibility.get("summary", "Analysis failed"),
                "key_strengths": key_strengths,
                "potential_concerns": potential_concerns,
                "strategic_recommendations": strategic_recommendations,
                "detailed_analysis": {
                    "industry_alignment": compatibility.get("industry_alignment", {"score": 0, "justification": "Analysis failed"}),
                    "stage_match": compatibility.get("stage_match", {"score": 0, "justification": "Analysis failed"}),
                    "funding_fit": compatibility.get("funding_fit", {"score": 0, "justification": "Analysis failed"}),
                    "business_model_alignment": compatibility.get("business_model_alignment", {"score": 0, "justification": "Analysis failed"}),
                    "strategic_alignment": compatibility.get("strategic_alignment", {"score": 0, "justification": "Analysis failed"}),
                    "team_fit": compatibility.get("team_fit", {"score": 0, "justification": "Analysis failed"}),
                    "market_alignment": compatibility.get("market_alignment", {"score": 0, "justification": "Analysis failed"}),
                    "vision_compatibility": compatibility.get("vision_compatibility", {"score": 0, "justification": "Analysis failed"}),
                    "risk_profile_match": compatibility.get("risk_profile_match", {"score": 0, "justification": "Analysis failed"}),
                    "growth_trajectory_fit": compatibility.get("growth_trajectory_fit", {"score": 0, "justification": "Analysis failed"})
                },
                "investor_profile": investor
            })

        # Sort by match score descending
        results.sort(key=lambda x: x["match_score"], reverse=True)

        # Limit to top N if specified
        if top_n and isinstance(top_n, int) and top_n > 0:
            results = results[:top_n]

        return results

    def generate_match_report(
        self,
        founder_data: Dict[str, Any],
        match_results: List[Dict[str, Any]],
        output_format: str = 'markdown'
    ) -> str:
        """
        Generate a comprehensive report of the matching results

        Args:
            founder_data: Dictionary containing founder information
            match_results: The results from match_founders_with_investors
            output_format: 'markdown' or 'html'

        Returns:
            String containing the formatted report
        """
        timestamp = datetime.now().strftime("%B %d, %Y - %H:%M")

        if output_format == 'markdown':
            report = f"# Founder-Investor Match Report\n\n"
            report += f"**Generated:** {timestamp}\n\n"

            report += "## Startup Profile\n\n"
            report += f"**Company:** {founder_data.get('name', 'Unnamed Startup')}\n"
            report += f"**Industry:** {founder_data.get('industry', 'N/A')}\n"
            report += f"**Stage:** {founder_data.get('stage', 'N/A')}\n"
            report += f"**Funding Required:** \\${founder_data.get('funding_required', 'N/A')}\n"
            report += f"**Business Model:** {founder_data.get('business_model', 'N/A')}\n\n"

            report += "## Top Investor Matches\n\n"

            for i, match in enumerate(match_results, 1):
                report += f"### {i}. {match['investor_name']} - Match Score: {match['match_score']:.1f}%\n\n"
                report += f"**Match Summary:**\n{match['match_summary']}\n\n"

                report += "**Key Strengths:**\n"
                report += f"{match['key_strengths']}\n\n"

                report += "**Potential Concerns:**\n"
                report += f"{match['potential_concerns']}\n\n"

                report += "**Strategic Recommendations:**\n"
                report += f"{match['strategic_recommendations']}\n\n"

                report += "**Dimensional Analysis:**\n\n"

                # Create a table for the dimensional analysis
                report += "| Dimension | Score | Justification |\n"
                report += "|-----------|-------|---------------|\n"

                for dim, details in match['detailed_analysis'].items():
                    dim_name = dim.replace('_', ' ').title()
                    score = details['score']
                    justification = details['justification']
                    report += f"| {dim_name} | {score:.1f} | {justification} |\n"

                report += "\n---\n\n"

        elif output_format == 'html':
            # HTML implementation would go here
            report = "<html><body><h1>HTML report generation not implemented yet</h1></body></html>"

        else:
            report = "Unsupported output format requested."

        return report

    def analyze_investor_compatibility_matrix(self, match_results: List[Dict[str, Any]]) -> pd.DataFrame:
        """
        Create a compatibility matrix to visualize strengths and weaknesses across investors

        Args:
            match_results: The results from match_founders_with_investors

        Returns:
            Pandas DataFrame with dimensional scores for each investor
        """
        # Extract the dimensions and scores
        data = []
        for match in match_results:
            row = {'investor': match['investor_name'], 'overall_score': match['match_score']}

            for dim, details in match['detailed_analysis'].items():
                row[dim] = details['score']

            data.append(row)

        # Create a DataFrame
        df = pd.DataFrame(data)

        # Set the investor name as the index
        if 'investor' in df.columns:
            df.set_index('investor', inplace=True)

        return df

    def recommend_investor_approach(self, investor_match: Dict[str, Any]) -> str:
        """
        Generate a tailored approach strategy for a specific investor based on the match analysis

        Args:
            investor_match: A single match result from the match_founders_with_investors output

        Returns:
            String containing approach recommendations
        """
        investor_name = investor_match['investor_name']
        profile = investor_match['investor_profile']
        analysis = investor_match['detailed_analysis']

        # Identify strengths (dimensions with scores > 75)
        strengths = []
        for dim, details in analysis.items():
            if details['score'] > 75:
                strengths.append(dim.replace('_', ' ').title())

        # Identify weaknesses (dimensions with scores < 50)
        weaknesses = []
        for dim, details in analysis.items():
            if details['score'] < 50:
                weaknesses.append((dim.replace('_', ' ').title(), details['justification']))

        # Generate the approach strategy
        strategy = f"## Approach Strategy for {investor_name}\n\n"

        # Lead with strengths
        strategy += "### Lead With These Strengths\n\n"
        if strengths:
            for strength in strengths:
                strategy += f"- **{strength}**: Emphasize this as a key selling point\n"
        else:
            strategy += "- No particularly strong match points identified. Consider a more general approach.\n"

        # Address potential concerns
        strategy += "\n### Prepare to Address These Concerns\n\n"
        if weaknesses:
            for weak, justification in weaknesses:
                strategy += f"- **{weak}**: {justification}\n"
        else:
            strategy += "- No significant concerns identified.\n"

        # Personalization tips
        strategy += "\n### Personalization Tips\n\n"

        # Reference past investments if available
        if 'past_investments' in profile and profile['past_investments']:
            strategy += "- Reference their investments in: " + ", ".join(profile['past_investments'][:3]) + "\n"

        # Reference their thesis
        if 'investment_thesis' in profile and profile['investment_thesis']:
            strategy += f"- Connect your vision to their thesis: \"{profile['investment_thesis'][:100]}...\"\n"

        # Value-add alignment
        if 'value_add' in profile and profile['value_add']:
            strategy += f"- Request specific help with: {profile['value_add']}\n"

        return strategy

class FounderInvestorMatchingPlatform:
    """
    Comprehensive platform for founder-investor matching with additional analysis tools,
    reporting capabilities, and integration features.
    """

    def __init__(self, api_key: Optional[str] = None, model_name: str = 'gemini-1.5-pro'):
        """
        Initialize the platform with the core matcher and additional features

        Args:
            api_key: Gemini API key
            model_name: The specific model to use
        """
        self.matcher = EnhancedFounderInvestorMatcher(api_key, model_name)
        self.match_history = []
        self.investor_database = {}
        self.founder_database = {}

    def add_investors_to_database(self, investors: List[Dict[str, Any]]) -> None:
        """
        Add investors to the platform's database for future matching

        Args:
            investors: List of investor profiles
        """
        for investor in investors:
            if 'name' in investor:
                self.investor_database[investor['name']] = investor

    def add_founders_to_database(self, founders: List[Dict[str, Any]]) -> None:
        """
        Add founders to the platform's database for future matching

        Args:
            founders: List of founder profiles
        """
        for founder in founders:
            if 'name' in founder:
                self.founder_database[founder['name']] = founder

    def find_match(
        self,
        founder_name: str,
        top_n: int = 3,
        custom_investors: List[Dict[str, Any]] = None
    ) -> Dict[str, Any]:
        """
        Find matches for a specific founder in the database

        Args:
            founder_name: Name of the founder to match
            top_n: Number of top matches to return
            custom_investors: Optional custom list of investors to match against

        Returns:
            Dictionary with match results and analysis
        """
        if founder_name not in self.founder_database:
            raise ValueError(f"Founder '{founder_name}' not found in database")

        founder_data = self.founder_database[founder_name]

        # Determine which investors to match against
        if custom_investors is not None:
            investors_to_match = custom_investors
        else:
            investors_to_match = list(self.investor_database.values())

        # Perform the matching
        match_results = self.matcher.match_founders_with_investors(
            founder_data,
            investors_to_match,
            top_n=top_n
        )

        # Generate additional analysis
        compatibility_matrix = self.matcher.analyze_investor_compatibility_matrix(match_results)

        # Generate the match report
        match_report = self.matcher.generate_match_report(founder_data, match_results)

        # Create approach strategies for the top matches
        approach_strategies = {}
        for match in match_results:
            investor_name = match['investor_name']
            approach_strategies[investor_name] = self.matcher.recommend_investor_approach(match)

        # Store the match results in history
        match_record = {
            'timestamp': datetime.now().isoformat(),
            'founder': founder_name,
            'investors_matched': len(investors_to_match),
            'top_matches': [m['investor_name'] for m in match_results],
            'match_results': match_results
        }
        self.match_history.append(match_record)

        # Return the comprehensive results
        return {
            'founder': founder_data,
            'match_results': match_results,
            'compatibility_matrix': compatibility_matrix,
            'match_report': match_report,
            'approach_strategies': approach_strategies
        }

    def batch_match_founders(
        self,
        founder_names: List[str],
        investors_per_founder: int = 3
    ) -> Dict[str, Dict[str, Any]]:
        """
        Perform batch matching for multiple founders

        Args:
            founder_names: List of founder names to match
            investors_per_founder: Number of investors to match per founder

        Returns:
            Dictionary mapping founder names to their match results
        """
        batch_results = {}

        for founder_name in founder_names:
            try:
                match_result = self.find_match(founder_name, top_n=investors_per_founder)
                batch_results[founder_name] = match_result
            except Exception as e:
                print(f"Error matching founder '{founder_name}': {e}")
                batch_results[founder_name] = {"error": str(e)}

        return batch_results

    def generate_investor_insights(self, investor_name: str) -> Dict[str, Any]:
        """
        Generate insights about a specific investor based on past matches

        Args:
            investor_name: Name of the investor to analyze

        Returns:
            Dictionary with insights about the investor
        """
        if investor_name not in self.investor_database:
            raise ValueError(f"Investor '{investor_name}' not found in database")

        investor_profile = self.investor_database[investor_name]

        # Find all matches involving this investor
        relevant_matches = []
        for match_record in self.match_history:
            for match in match_record['match_results']:
                if match['investor_name'] == investor_name:
                    match_with_founder = {
                        'founder': match_record['founder'],
                        'score': match['match_score'],
                        'timestamp': match_record['timestamp'],
                        'detailed_analysis': match['detailed_analysis']
                    }
                    relevant_matches.append(match_with_founder)

        # Analyze match patterns
        if relevant_matches:
            # Calculate average scores across dimensions
            dimension_scores = {
                'industry_alignment': [],
                'stage_match': [],
                'funding_fit': [],
                'business_model_alignment': [],
                'strategic_alignment': [],
                'team_fit': [],
                'market_alignment': [],
                'vision_compatibility': [],
                'risk_profile_match': [],
                'growth_trajectory_fit': []
            }

            for match in relevant_matches:
                for dim, values in dimension_scores.items():
                    if dim in match['detailed_analysis']:
                        values.append(match['detailed_analysis'][dim]['score'])

            # Calculate averages for each dimension
            dimension_averages = {}
            for dim, scores in dimension_scores.items():
                if scores:
                    dimension_averages[dim] = sum(scores) / len(scores)
                else:
                    dimension_averages[dim] = 0

            # Find strongest and weakest dimensions
            sorted_dimensions = sorted(dimension_averages.items(), key=lambda x: x[1], reverse=True)
            strongest_dimensions = sorted_dimensions[:3]
            weakest_dimensions = sorted_dimensions[-3:]

            # Calculate match trend over time
            matches_by_time = sorted(relevant_matches, key=lambda x: x['timestamp'])
            match_trend = [
                {'timestamp': m['timestamp'], 'score': m['score']}
                for m in matches_by_time
            ]

            insights = {
                'investor_profile': investor_profile,
                'match_count': len(relevant_matches),
                'average_match_score': sum(m['score'] for m in relevant_matches) / len(relevant_matches),
                'dimension_averages': dimension_averages,
                'strongest_dimensions': strongest_dimensions,
                'weakest_dimensions': weakest_dimensions,
                'match_trend': match_trend,
                'founder_matches': [m['founder'] for m in relevant_matches]
            }
        else:
            insights = {
                'investor_profile': investor_profile,
                'match_count': 0,
                'message': "No match history found for this investor"
            }

        return insights

    def export_match_data(self, format: str = 'json') -> Any:
        """
        Export all match data in the specified format

        Args:
            format: 'json', 'csv', or 'pandas'

        Returns:
            Data in the specified format
        """
        if format == 'json':
            return json.dumps(self.match_history, indent=2)

        elif format == 'pandas':
            # Create a flattened dataframe
            records = []
            for match_record in self.match_history:
                founder = match_record['founder']
                timestamp = match_record['timestamp']

                for match in match_record['match_results']:
                    record = {
                        'timestamp': timestamp,
                        'founder': founder,
                        'investor': match['investor_name'],
                        'match_score': match['match_score']
                    }

                    # Add dimension scores
                    for dim, details in match['detailed_analysis'].items():
                        record[f'{dim}_score'] = details['score']

                    records.append(record)

            return pd.DataFrame(records)

        elif format == 'csv':
            df = self.export_match_data(format='pandas')
            return df.to_csv(index=False)

        else:
            raise ValueError(f"Unsupported export format: {format}")

    def get_market_insights(self) -> Dict[str, Any]:
        """
        Analyze patterns across all matches to generate market insights

        Returns:
            Dictionary with market insights
        """
        if not self.match_history:
            return {"message": "No match data available for analysis"}

        # Prepare data for analysis
        all_matches = []
        for record in self.match_history:
            founder = record['founder']
            founder_industry = self.founder_database.get(founder, {}).get('industry', 'Unknown')
            founder_stage = self.founder_database.get(founder, {}).get('stage', 'Unknown')

            for match in record['match_results']:
                all_matches.append({
                    'founder': founder,
                    'industry': founder_industry,
                    'stage': founder_stage,
                    'investor': match['investor_name'],
                    'score': match['match_score']
                })

        matches_df = pd.DataFrame(all_matches)

        # Industry analysis
        industry_analysis = {}
        if 'industry' in matches_df.columns:
            avg_scores_by_industry = matches_df.groupby('industry')['score'].mean().sort_values(ascending=False)
            match_counts_by_industry = matches_df.groupby('industry').size().sort_values(ascending=False)

            industry_analysis = {
                'average_scores': avg_scores_by_industry.to_dict(),
                'match_counts': match_counts_by_industry.to_dict(),
                'top_industries': avg_scores_by_industry.head(3).index.tolist(),
                'bottom_industries': avg_scores_by_industry.tail(3).index.tolist()
            }

        # Stage analysis
        stage_analysis = {}
        if 'stage' in matches_df.columns:
            avg_scores_by_stage = matches_df.groupby('stage')['score'].mean().sort_values(ascending=False)
            match_counts_by_stage = matches_df.groupby('stage').size().sort_values(ascending=False)

            stage_analysis = {
                'average_scores': avg_scores_by_stage.to_dict(),
                'match_counts': match_counts_by_stage.to_dict(),
                'top_stages': avg_scores_by_stage.head(3).index.tolist(),
                'bottom_stages': avg_scores_by_stage.tail(3).index.tolist()
            }

        # Investor analysis
        investor_analysis = {}
        if 'investor' in matches_df.columns:
            avg_scores_by_investor = matches_df.groupby('investor')['score'].mean().sort_values(ascending=False)
            match_counts_by_investor = matches_df.groupby('investor').size().sort_values(ascending=False)

            investor_analysis = {
                'average_scores': avg_scores_by_investor.to_dict(),
                'match_counts': match_counts_by_investor.to_dict(),
                'most_compatible_investors': avg_scores_by_investor.head(3).index.tolist(),
                'least_compatible_investors': avg_scores_by_investor.tail(3).index.tolist(),
                'most_active_investors': match_counts_by_investor.head(3).index.tolist()
            }

        # Return comprehensive insights
        return {
            'industry_analysis': industry_analysis,
            'stage_analysis': stage_analysis,
            'investor_analysis': investor_analysis,
            'total_matches_analyzed': len(all_matches),
            'average_match_score': matches_df['score'].mean() if 'score' in matches_df.columns else 0,
            'generated_at': datetime.now().isoformat()
        }

# Example usage with enhanced functionality
def main():
    """Run a comprehensive demo of the founder-investor matching platform"""

    # Get API key from environment or user input
    gemini_api_key = ""  # Example only

    try:
        # Initialize the platform
        print("Initializing the FounderInvestorMatchingPlatform...")
        platform = FounderInvestorMatchingPlatform(api_key=gemini_api_key)

        # Load sample data
        founder_data = get_sample_founder_data()
        investors_data = get_sample_investors_data()

        # Add to platform database
        platform.add_founders_to_database([founder_data])
        platform.add_investors_to_database(investors_data)

        print(f"\nPlatform initialized with {len(platform.founder_database)} founders and {len(platform.investor_database)} investors")

        # Define the Gradio interface
        def run_match(founder_name, top_n):
            match_results = platform.find_match(founder_name, top_n=top_n)
            return match_results['match_report']

        def export_data(format):
            return platform.export_match_data(format=format)

        def get_insights():
            return platform.get_market_insights()

        # Create the Gradio interface
        with gr.Blocks() as demo:
            gr.Markdown("# Founder-Investor Matching Platform")

            with gr.Tab("Match Founders"):
                founder_name = gr.Textbox(label="Founder Name")
                top_n = gr.Number(label="Top N Matches", value=3)
                match_button = gr.Button("Run Match")
                match_output = gr.Textbox(label="Match Report")

                match_button.click(run_match, inputs=[founder_name, top_n], outputs=match_output)

            with gr.Tab("Export Data"):
                export_format = gr.Dropdown(choices=["json", "csv", "pandas"], label="Export Format")
                export_button = gr.Button("Export Data")
                export_output = gr.File(label="Exported Data")

                export_button.click(export_data, inputs=export_format, outputs=export_output)

            with gr.Tab("Market Insights"):
                insights_button = gr.Button("Get Market Insights")
                insights_output = gr.JSON(label="Market Insights")

                insights_button.click(get_insights, outputs=insights_output)

        # Launch the Gradio interface
        demo.launch()

    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

# Example of enhanced founder data structure
def get_sample_founder_data() -> Dict[str, Any]:
    return {
        "name": "TechInnovate AI",
        "industry": "Artificial Intelligence / Machine Learning",
        "stage": "Seed",
        "funding_required": 750000,
        "traction": "5,000 monthly active users, \\$15,000 MRR, 12% MoM growth",
        "business_model": "B2B SaaS - Subscription",
        "target_market": "Mid-sized enterprises in healthcare, finance, and retail",
        "product_status": "MVP with 15 paying customers",
        "team_background": "Founded by ex-Google AI researcher and Stanford CS PhD",
        "team_experience": "Combined 12 years in AI/ML, previous startup exit of \\$8M",
        "vision": "Democratize access to AI tools for businesses of all sizes",
        "usp": "Custom AI models with 60% less training data than competitors",
        "growth_strategy": "Land and expand within enterprise accounts, strategic partnerships",
        "market_size": "\\$15B TAM, growing at 35% CAGR",
        "market_trend": "Increasing demand for specialized AI solutions in regulated industries",
        "competitive_landscape": "Fragmented market with no dominant player in our niche",
        "exit_strategy": "Acquisition by major cloud provider or enterprise software company within 5-7 years",
        "challenges": "Scaling sales team, managing increasing cloud compute costs"
    }

# Enhanced investor data structure
def get_sample_investors_data() -> List[Dict[str, Any]]:
    return [
        {
            "name": "Alpha Ventures",
            "preferred_industries": ["AI", "Machine Learning", "SaaS", "Enterprise Software"],
            "investment_range": {"min": 500000, "max": 2000000},
            "preferred_stages": ["Seed", "Pre-seed"],
            "investment_thesis": "Investing in AI-driven solutions that transform traditional industries with a focus on enterprise applications",
            "investment_style": "Hands-on, weekly check-ins, active board participation",
            "past_investments": ["DataRobot", "ScaleAI", "Hugging Face"],
            "value_add": "AI talent recruitment, enterprise sales connections, cloud credits",
            "geographic_focus": "North America, Europe",
            "roi_timeline": "5-7 years",
            "risk_tolerance": "Moderate - expects 1 in 5 investments to drive returns",
            "decision_factors": "Team expertise, technology differentiation, market opportunity",
            "board_requirements": "Observer seat at minimum",
            "follow_on_potential": "Up to \\$5M in follow-on funding for top performers",
            "constraints": "No hardware companies, no pure consumer plays"
        },
        {
            "name": "Beta Capital",
            "preferred_industries": ["Fintech", "Blockchain", "Crypto", "AI in Finance"],
            "investment_range": {"min": 1000000, "max": 5000000},
            "preferred_stages": ["Series A"],
            "investment_thesis": "Supporting financial technology innovation with a focus on infrastructure and B2B solutions",
            "investment_style": "Capital-focused, quarterly reviews, limited operational involvement",
            "past_investments": ["Stripe", "Plaid", "Coinbase"],
            "value_add": "Financial services partnerships, regulatory guidance, M&A support",
            "geographic_focus": "Global",
            "roi_timeline": "4-6 years",
            "risk_tolerance": "Conservative - focuses on proven business models",
            "decision_factors": "Revenue growth, unit economics, regulatory compliance",
            "board_requirements": "Board seat for investments over \\$2M",
            "follow_on_potential": "Participates pro-rata in future rounds",
            "constraints": "Must have \\$500K+ ARR for Series A"
        },
        {
            "name": "Gamma Partners",
            "preferred_industries": ["Healthcare", "Biotech", "AI in Healthcare"],
            "investment_range": {"min": 250000, "max": 1000000},
            "preferred_stages": ["Pre-seed", "Seed"],
            "investment_thesis": "Backing early-stage startups using technology to improve healthcare outcomes",
            "investment_style": "Founder-friendly, supportive, monthly advisory calls",
            "past_investments": ["Tempus", "Recursion", "PathAI"],
            "value_add": "Clinical trial design support, FDA approval guidance, hospital network access",
            "geographic_focus": "North America",
            "roi_timeline": "7-10 years",
            "risk_tolerance": "High - willing to back moonshot ideas",
            "decision_factors": "Novel IP, clinical value proposition, team scientific credibility",
            "board_requirements": "No formal requirements at seed stage",
            "follow_on_potential": "Selective follow-on strategy",
            "constraints": "Requires at least one healthcare domain expert on founding team"
        },
        {
            "name": "Delta Seed Fund",
            "preferred_industries": ["AI", "Data Infrastructure", "Developer Tools", "Enterprise SaaS"],
            "investment_range": {"min": 400000, "max": 1500000},
            "preferred_stages": ["Pre-seed", "Seed"],
            "investment_thesis": "Supporting technical founders building foundational technology and tools for the AI-driven economy",
            "investment_style": "Technical support, founder-centric, biweekly office hours",
            "past_investments": ["Databricks", "Snorkel AI", "Weights & Biases"],
            "value_add": "Technical architecture review, open source strategy, dev community building",
            "geographic_focus": "North America, Europe, Israel",
            "roi_timeline": "5-8 years",
            "risk_tolerance": "Moderate-High - believes in technical differentiation",
            "decision_factors": "Technical innovation, IP potential, founder technical credibility",
            "board_requirements": "Advisory role only at seed stage",
            "follow_on_potential": "Reserves 2x initial investment for follow-on",
            "constraints": "Founders must have technical background"
        },
        {
            "name": "Epsilon Growth Capital",
            "preferred_industries": ["AI Applications", "Vertical SaaS", "Industry-Specific Software"],
            "investment_range": {"min": 1000000, "max": 3000000},
            "preferred_stages": ["Seed", "Series A"],
            "investment_thesis": "Scaling proven AI applications with industry-specific focus to achieve rapid market penetration",
            "investment_style": "Growth-focused, operational support, sales acceleration",
            "past_investments": ["Gong.io", "UiPath", "Canva"],
            "value_add": "Enterprise sales strategy, pricing optimization, go-to-market support",
            "geographic_focus": "North America, Europe, Australia",
            "roi_timeline": "4-6 years",
            "risk_tolerance": "Moderate - prefers proven product-market fit",
            "decision_factors": "Revenue growth rate, CAC/LTV, market expansion potential",
            "board_requirements": "Board seat for investments over \\$1.5M",
            "follow_on_potential": "Strong follow-on capabilities up to \\$10M total per company",
            "constraints": "Must demonstrate clear product-market fit with at least 10 paying customers"
        }
    ]

if __name__ == "__main__":
    main()


Initializing the FounderInvestorMatchingPlatform...

Platform initialized with 1 founders and 5 investors
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://60b8365255c4ac53cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
